<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Computação em Nuvem III
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)<br>
Aluna: [Maria Luiza Cordeiro](https://www.linkedin.com/in/malu-analise-de-dados/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Apache Spark;</li>
  <li>Data Wrangling com Spark.</li>
</ol>

---

# **Exercícios**

## 1\. Apache Spark

Replique as atividades do item 2.1 e 2.2 para instalar e configurar um cluster Apache Spark na máquina virtual do Google Colab.

### **1.1. Instalação**

In [1]:
%%capture
!apt-get remove openjdk*
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

!tar xf spark-3.3.2-bin-hadoop3.tgz

In [3]:
!pip install pyspark

In [4]:
!pip install findspark

### **1.2. Configuração**

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

In [6]:
import findspark

findspark.init()

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("pyspark-notebook").getOrCreate()

## 2\. Data Wrangling

A base de dados presente neste [link](https://www.kaggle.com/datasets/bank-of-england/a-millennium-of-macroeconomic-data) contem dados macroeconômicos sobre o Reino Unido desde o século 13.

**2.1\. Data**

Faça o download dos dados utilizando a máquina virutal do Google Colab com o código abaixo.

In [8]:
!wget -q "https://raw.githubusercontent.com/cluster-apps-on-docker/spark-standalone-cluster-on-docker/master/build/workspace/data/uk-macroeconomic-data.csv" -O "uk-macroeconomic-data.csv"

In [9]:
data = spark.read.csv(path="uk-macroeconomic-data.csv", sep=",", header=True)

In [10]:
data.show()

+-----------+------------------------------------+-----------------------------------+-------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------------------+----------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+--------------------+--------------------+--------------------------+-------------------------------------------------+--------------------+--------------------+---------------------------------------+-------------------------------+---------------------------------+------------------+--------------------+----------+-----------------+---------------------------+------------------------------

In [11]:
data.printSchema()

root
 |-- Description: string (nullable = true)
 |-- Real GDP of England at market prices: string (nullable = true)
 |-- Real GDP of England at factor cost : string (nullable = true)
 |-- Real UK GDP at market prices, geographically-consistent estimate based on post-1922 borders: string (nullable = true)
 |-- Real UK GDP at factor cost, geographically-consistent estimate based on post-1922 borders: string (nullable = true)
 |-- Index of real UK GDP at factor cost - based on changing political boundaries, : string (nullable = true)
 |-- Composite estimate of English and (geographically-consistent) UK real GDP at factor cost: string (nullable = true)
 |-- HP-filter of log of real composite estimate of English and UK real GDP at factor cost: string (nullable = true)
 |-- Real UK gross disposable national income at market prices, constant border estimate: string (nullable = true)
 |-- Real consumption: string (nullable = true)
 |-- Real investment: string (nullable = true)
 |-- Stockbuildi

**2.2. Wrangling**

Processe os dados para que a base de dados final apresente os valores da taxa de desemprego (`Unemployment rate`) e população (`Population (GB+NI)`) estejam ordenados por ano decrescente:

```csv
year,population,unemployment_rate
...,...,...
```

Para isso, utilize:

 - Pandas

In [12]:
import pandas as pd

df = pd.read_csv("uk-macroeconomic-data.csv")

df = df[["Description", "Population (GB+NI)", "Unemployment rate"]]
df.columns = ["year", "population", "unemployment_rate"]

df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["population"] = pd.to_numeric(df["population"], errors="coerce")
df["unemployment_rate"] = pd.to_numeric(df["unemployment_rate"], errors="coerce")

df = df.dropna()

df = df.sort_values(by="year", ascending=False)

df = df.reset_index(drop=True)

print(df.head())

     year  population  unemployment_rate
0  2016.0     65573.0               4.90
1  2015.0     65110.0               5.38
2  2014.0     64597.0               6.18
3  2013.0     64106.0               7.61
4  2012.0     63705.0               7.97


 - PySpark

In [14]:
data_description = data.filter(data['Description'] == 'Units')

In [15]:
from pyspark.sql.functions import broadcast

In [18]:
data = data.join(other=broadcast(data_description), on=['Description'], how='left_anti')

In [21]:
from pyspark.sql.functions import col

df = data.select(
    col("Description").cast("int").alias("year"),
    col("`Population (GB+NI)`").cast("float").alias("population"),
    col("`Unemployment rate`").cast("float").alias("unemployment_rate")
)

df_ordered = df.orderBy(col("year").desc())

df_ordered.show()

+----+----------+-----------------+
|year|population|unemployment_rate|
+----+----------+-----------------+
|2016|   65573.0|              4.9|
|2015|   65110.0|             5.38|
|2014|   64597.0|             6.18|
|2013|   64106.0|             7.61|
|2012|   63705.0|             7.97|
|2011|   63285.0|             8.11|
|2010|   62759.0|             7.87|
|2009|   62260.0|             7.61|
|2008|   61824.0|             5.69|
|2007|   61319.0|             5.33|
|2006|   60827.0|             5.42|
|2005|   60413.0|             4.83|
|2004|   59950.0|             4.75|
|2003|   59637.0|             5.01|
|2002|   59366.0|             5.19|
|2001|   59113.0|              5.1|
|2000|   58886.0|             5.46|
|1999|   58684.0|             5.98|
|1998|   58475.0|             6.26|
|1997|   58314.0|             6.97|
+----+----------+-----------------+
only showing top 20 rows

